In [ ]:
# Import the libraries
import os 

import gym

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize

from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack

from huggingface_sb3 import load_from_hub, push_to_hub

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

import pickle


: 

In [ ]:

# Load the model
checkpoint = load_from_hub("ThomasSimonini/ppo-PongNoFrameskip-v4", "ppo-PongNoFrameskip-v4.zip")

# Because we using 3.7 on Colab and this agent was trained with 3.8 to avoid Pickle errors:
custom_objects = {
            "learning_rate": 0.0,
            "lr_schedule": lambda _: 0.0,
            "clip_range": lambda _: 0.0,
        }

model= PPO.load(checkpoint, custom_objects=custom_objects)

: 

In [ ]:
import gymnasium as gym
env = gym.make("ALE/Pong-v5", render_mode="rgb_array")

: 

In [ ]:
env = make_atari_env('PongNoFrameskip-v4', n_envs=1)
env = VecFrameStack(env, n_stack=4)

: 

In [ ]:

obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    
    frame = env.render(mode="rgb_array")
    plt.imshow(frame)
    plt.axis("off")
    clear_output(wait=True)

    plt.show()

    if dones[0]:
        break
env.close()

: 

In [ ]:
obs = env.reset()
frame = env.render(mode="rgb_array")
model.predict(obs)

: 

# Collect Dataset

In [ ]:


# Set the number of episodes to collect trajectories
n_episodes = 1_000

# Storage for trajectories
trajectories = []

# Run episodes to collect trajectories
for episode in range(n_episodes):
    obs = env.reset()
    episode_data = {
        "state": [],
        "next_state": [],
        "actions": [],
        "rewards": [],
        "dones": []
    }

    done = [False]
    for i in range(10_000_000):
        action, _states = model.predict(obs)
        next_obs, reward, done, info = env.step(action)
        

        # Store step data in trajectory
        episode_data["state"].append(obs)   # Store the stacked frames
        episode_data["next_state"].append(next_obs)   # Store the stacked next_frames
        episode_data["actions"].append(action)      # Store the selected action
        episode_data["rewards"].append(reward)      # Store the received reward
        episode_data["dones"].append(done)          # Store the done flag

        if done[0]:
            break
    
        obs = next_obs

    # Append episode data to trajectories
    trajectories.append(episode_data)

    if episode % 20 == 0:
        # Save trajectories to a file
        with open("/home/hice1/rmehta98/scratch/pong_trajectories.pkl", "wb") as f:
            pickle.dump(trajectories, f)

print("Trajectories saved to pong_trajectories.pkl")

# Close the environment
env.close()

: 

: 